1) Where did environment go?
2) How do I turn an AmlCompute into a ComputeConfiguration
3) What args are required for Parameterized command?
3a) Is "base_path" where the script is located?

In [ ]:
# Make workspace
%env AZURE_EXTENSION_DIR=/home/schrodinger/automl/sdk-cli-v2/src/cli/src
%env AZURE_ML_CLI_PRIVATE_FEATURES_ENABLED=true

# Prepare workspace, experiment, and compute

In [ ]:
import azure.ml
from azure.ml import MLClient

subscription_id = '381b38e9-9840-4719-a5a0-61d9585e1e91'
resource_group_name = 'lesam_centraluseuap_rg'
workspace_name = "lesam-master"
experiment_name = "MySdkV2SweepJobExperiment"

client = MLClient(subscription_id, resource_group_name, default_workspace_name=workspace_name)

client

In [ ]:
# Make Compute
# choose a name for your cluster
from azure.ml.entities import AmlCompute
cluster_name = "cpu-cluster"

try:
    compute_target = client.compute.get(cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute = AmlCompute(
        name=cpu_cluster_name, size="STANDARD_D13_V2",
        min_instances=0, max_instances=3,
        idle_time_before_scale_down=120
    )

    # create the cluster
    client.compute.create(compute)

# use get_status() to get a detailed status for the current cluster. 
compute_target

# Prepare Sweep Job

In [ ]:
# Prepare Early Termination Policy
from azure.ml._restclient.v2021_03_01_preview.models import MedianStoppingPolicy
early_termination_policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5)

In [ ]:
# Prepare search space
from azure.ml.entities import Choice, Uniform, Randint
search_space = {
    "y1": Choice(["foo", "bar", "baz"]),
    "y2": Uniform(0, 1),
    "y3": Randint(999)
}

In [ ]:
# Make SweepJob
from azure.ml.entities import SweepJob, ParameterizedCommand
from azure.ml._restclient.v2021_03_01_preview.models import Objective

objective = Objective(goal="Maximize", primary_metric="Accuracy")
trial = ParameterizedCommand(
    base_path = ".",
    command = 'python example_script.py --x1 $AZUREML_SWEEP_y1 --x2 $AZUREML_SWEEP_y2 --x3 $AZUREML_SWEEP_y3'
)

In [ ]:
sweep_job = SweepJob(
    algorithm = "Random",
    search_space = search_space,
    objective = objective,
    trial = trial,
    early_termination = early_termination_policy,
    max_concurrent_trials = 2,
    max_total_trials = 2,
    timeout_minutes = 60,
)

In [ ]:
# Submit
created_job = client.jobs.create_or_update(sweep_job)
created_job